In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import nltk

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
df = pd.read_csv('data/spam.csv',encoding='ISO-8859-1')

In [7]:
df = df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [8]:
df = df.rename(columns={'v1':'result','v2':'sms'})

In [9]:
df.head()

,result,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df['sms']
y = df['result']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=101)

In [13]:
tfidf = TfidfVectorizer(stop_words='english')

In [14]:
tfidf.fit(X_train)

TfidfVectorizer(stop_words='english')

In [15]:
X_train_tfidf = tfidf.transform(X_train)
X_train_tfidf.shape

(4736, 7731)

In [17]:
X_test_tfidf = tfidf.transform(X_test)

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rfc = RandomForestClassifier(max_features='sqrt')

In [20]:
rfc.fit(X_train_tfidf,y_train)

RandomForestClassifier()

In [22]:
predict = rfc.predict(X_test_tfidf)

In [26]:
from sklearn.pipeline import Pipeline

In [28]:
pipe = Pipeline(steps=[('tfidf',TfidfVectorizer()),('rfc',RandomForestClassifier())])

In [29]:
pipe.fit(X,y)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('rfc', RandomForestClassifier())])

In [34]:
print(pipe.predict(['Hello dear, its been 3 weeks'])[0])

ham


In [35]:
from sklearn.naive_bayes import MultinomialNB

In [36]:
nb = MultinomialNB()

In [37]:
nb.fit(X_train_tfidf,y_train)

MultinomialNB()

In [46]:
pipe_nb = Pipeline(steps=[('tfidf',TfidfVectorizer()),('nb',MultinomialNB())])
pipe_nb.fit(X,y)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('nb', MultinomialNB())])

In [55]:
print(pipe_nb.predict(['You have won!'])[0])

ham


In [52]:
from sklearn.metrics import confusion_matrix

In [50]:
y_pred = nb.predict(X_test_tfidf)

In [53]:
cm = confusion_matrix(y_test,y_pred)

In [54]:
cm

array([[735,   0],
       [ 22,  79]], dtype=int64)